In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import numpy as np

from pdg import PDG
from dist import RawJointDist as RJD
from lib import A,B,C,D
from lib.square import with_indeps, consist_with_P, P

In [3]:
# μ_opt = with_indeps.optimize_score(gamma=0.001, tol=1E-30)
φ = consist_with_P.factor_product(repr='atomic')
μCP_opt1, μCP_iter1 = consist_with_P.optimize_score(gamma=1, store_iters=True)
μCP_opt0, μCP_iter0 = consist_with_P.optimize_score(gamma=0, store_iters=True)

In [4]:
μCP_GS_ord, μCP_iterGSo = consist_with_P.iter_GS_ordered(store_iters=True)
μCP_GS_β, μCP_iterGSβ = consist_with_P.iter_GS_beta(store_iters=True)

It should be the case that any unweighted PDG $M$ will have the same distribution $q_{\gamma=0}$ that minimizes its semantics $ [[ M ]]$ at $\gamma=1$, which is equal to its factor product $\Pr_{\varphi_{M}}$.

What GIbbs sampling do to $\mathit{Inc}$ and $\mathit{IDef}$?

So... let's set up a PCA to visualize what's happening. with both optimizations, $P$, and $\varphi$

In [5]:
SHAPE = φ.data.reshape(-1).shape

Ppt = np.array([P.data.reshape(*SHAPE)])
φpt = np.array([φ.data.reshape(*SHAPE)])
idx0pt = np.linspace(0, 1, len(μCP_iter0))
idx1pt = np.linspace(0, 1, len(μCP_iter1))

iter0pt = np.array(μCP_iter0)
iter1pt = np.array(μCP_iter1)
bigmatrix = np.vstack([Ppt, φpt, iter0pt, iter1pt])

def iitransform(M, μflatmat):
    return [(M.Inc(μ).real,M.IDef(μ).real) for μ in map(lambda d : RJD(d.reshape(*M.dshape), M.varlist), μflatmat)]

%matplotlib widget
%matplotlib widget

# Do for γ=0 (P)
plt.plot(*zip(*iitransform(consist_with_P, iter0pt[0:])), 'b', alpha=0.1)
plt.scatter(*zip(*iitransform(consist_with_P, iter0pt)), cmap='Purples', c=idx0pt, alpha=0.3)
Xs,Ys = zip(*iitransform(consist_with_P, [Ppt])); plt.plot(Xs,Ys, '*w', markersize=15,alpha=0.8)
Xs,Ys = zip(*iitransform(consist_with_P, [μCP_opt0.data])); plt.plot(Xs,Ys, 'xb', markersize=15,alpha=0.8)
plt.annotate

# Do for γ=1 (φ)
plt.plot(*zip(*iitransform(consist_with_P, iter1pt[0:])), 'r', alpha=0.1)
plt.scatter(*zip(*iitransform(consist_with_P, iter1pt)), cmap='Oranges', c=idx0pt, alpha=0.3)
Xs,Ys = zip(*iitransform(consist_with_P, [φpt])); plt.plot(Xs,Ys, 'or', markersize=15,alpha=0.8)
Xs,Ys = zip(*iitransform(consist_with_P, [μCP_opt1.data])); plt.plot(Xs,Ys, 'xr', markersize=15,alpha=0.8)

#now for GS
plt.plot(*zip(*iitransform(consist_with_P, μCP_iterGSo)), 'g', alpha=0.1)
plt.scatter(*zip(*iitransform(consist_with_P, μCP_iterGSo)), cmap='Greens', c=np.linspace(0,1,len(μCP_iterGSo)), alpha=0.3)
Xs,Ys = zip(*iitransform(consist_with_P, [μCP_GS_ord.data])); plt.plot(Xs,Ys, '+g', markersize=15,alpha=0.8)

plt.plot(*zip(*iitransform(consist_with_P, μCP_iterGSβ)), 'k', alpha=0.1)
plt.scatter(*zip(*iitransform(consist_with_P, μCP_iterGSβ)), cmap='Greys', c=np.linspace(0,1,len(μCP_iterGSβ)), alpha=0.3)
Xs,Ys = zip(*iitransform(consist_with_P, [μCP_GS_β.data])); plt.plot(Xs,Ys, 'xk', markersize=15,alpha=0.8)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Weirld, that the optimziation does not quite get either the oange dot (the distribution $P$ which is supposed to be the # of factors), and also that the blue one does not minimize Inc. I need to make sure the gradient is correct.... and then implement accelerated gradient descent properly?

Questions:
 1. Why is the high density of orange points not where the final red X lies?
 3. Why is the factor product (orange circle) so far from the red X?
 2. Why is the blue X so high in Inc?

# Debugging the Gradient

In [9]:
M = consist_with_P
mscore = M._build_fast_scorer(gamma=0)

mscore(μCP_GS_β.data)[0], mscore(μCP_opt0.data)[0] # uh-oh...

(0.0, 1.9658301999899563)

Clearly the optimization isn't working properly. But this should be a convex function... let's look at the supposed gradients.

In [10]:
(mscore(μCP_GS_β.data)[1]**2).sum(), (mscore(μCP_opt0.data)[1]**2).sum()

(2.11677672154621e-24, 7983.54056253065)

So the gradient of the optimal is much larger... does it point towards the optimum?

In [11]:
xopt = μCP_GS_β.data.reshape(-1)
x = μCP_opt0.data.reshape(-1)
gradx = mscore(x)[1]

vec = (xopt - x)
vec /= np.sqrt((vec*vec).sum())
ϵ = 0.001

# np.allclose(vec.reshape(-1).dot(vec.reshape(-1)), (vec*vec).sum().reshape(-1))  #True
print("score @ x: %.3f;\t score @ x + ϵ·grad: %.3f;\t  score @x + ϵ·v: %.3f"%
      (mscore(x)[0], mscore(x + gradx*ϵ)[0], mscore(x + vec * ϵ)[0]))

score @ x: 1.966;	 score @ x + ϵ·grad: 9.204;	  score @x + ϵ·v: 1.954


Clearly, the gradient is wrong. Some possible reasons this could be the case:
 - I computed the gradient wrong in the math (unlikely because I was careful and I keep revisiting this)
 - The penalty is messing things up..